In [1]:
from time import sleep, time

from IPython.display import HTML, Latex, display
from tqdm import tqdm

from pyphysim.progressbar.progressbar import (
    DummyProgressbar,
    ProgressBarIPython,
    ProgressbarText,
    ProgressbarText2,
    ProgressbarText3,
)
from pyphysim.simulations.parameters import SimulationParameters
from pyphysim.simulations.results import Result, SimulationResults
from pyphysim.util.misc import pretty_time

In [2]:
# Create some dummy parameters (including two parameters set to be
# unpacked)
params = SimulationParameters()
params.add("extra", 2.3)
params.add("SNR", np.array([0, 3, 6, 9]))
params.add("bias", [1.2, 1.6])
params.set_unpack_parameter('SNR')
params.set_unpack_parameter('bias')
params.add("Name", "Some string")

sim_results = SimulationResults()
for p in params.get_unpacked_params_list():
    extra = p['extra']
    SNR = p['SNR']
    bias = p['bias']
    sim_results.append_result(
        Result.create('res1', Result.SUMTYPE, extra * SNR + bias))
    sim_results.append_result(
        Result.create('res2', Result.SUMTYPE, bias * SNR + extra))
sim_results.set_parameters(params)

# Now lets convert this SimulationResults object to a pandas
# DataFrame
df = sim_results.to_dataframe()

In [3]:
sim_results.to_dataframe()

,extra,SNR,bias,Name,res1,res2,runned_reps
0,2.3,0,1.2,Some string,1.2,2.3,None
1,2.3,0,1.6,Some string,1.6,2.3,None
2,2.3,3,1.2,Some string,8.1,5.9,None
3,2.3,3,1.6,Some string,8.5,7.1,None
4,2.3,6,1.2,Some string,15.0,9.5,None
5,2.3,6,1.6,Some string,15.4,11.9,None
6,2.3,9,1.2,Some string,21.9,13.1,None
7,2.3,9,1.6,Some string,22.3,16.7,None


In [4]:
print(sim_results)

SimulationResults: ['res1', 'res2']


In [5]:
(sim_results["res1"])

[Result -> res1: 1.2,
 Result -> res1: 1.6,
 Result -> res1: 8.1,
 Result -> res1: 8.5,
 Result -> res1: 14.999999999999998,
 Result -> res1: 15.399999999999999,
 Result -> res1: 21.9,
 Result -> res1: 22.3]

In [6]:
# pbar = ProgressbarText(10)
# pbar.start()
# for i in range(1, 11):
#     sleep(1)
#     pbar.progress(i)
#     print()
#     print(f"Elapsed time for {i} iterations: {pbar.elapsed_time}")
#     print(f"Remaining time: {pretty_time(pbar.get_eta_in_seconds(i))}")

In [7]:
# print(pbar.elapsed_time)
# print(pbar.get_eta_in_seconds(8))

# DummyProgressbar

In [8]:
tic = time()
num_iterations = 10000000
pbar = DummyProgressbar(num_iterations)
for i in range(1, num_iterations + 1):
    pbar.progress(i)
toc = time()
elapsed_time = toc - tic
print(elapsed_time)
print(f"Time per iteration: {elapsed_time/num_iterations}")

1.0966026782989502
Time per iteration: 1.0966026782989502e-07


# ProgressbarText

In [9]:
tic = time()
num_iterations = 5000000
pbar = ProgressbarText(num_iterations, message="Running some simulation")
for i in range(1, num_iterations + 1):
    pbar.progress(i)
toc = time()
elapsed_time = toc - tic
print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

------------ Running some simulation ------------1
    1    2    3    4    5    6    7    8    9    0
----0----0----0----0----0----0----0----0----0----0
**************************************************
Elapsed time:  2.33s
Time per iteration: 4.664907932281494e-07


# ProgressbarText2

In [10]:
tic = time()
num_iterations = 15000000
pbar = ProgressbarText2(num_iterations, message="{percent:.2f} {elapsed_time} {eta}")
for i in range(1, num_iterations + 1):
    pbar.progress(i)
toc = time()
elapsed_time = toc - tic
print()
print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

[**********************100%**********************]  100.00 6.92s 0.00s

Elapsed time:  6.92s
Time per iteration: 4.616387526194255e-07


# ProgressbarText3

In [11]:
tic = time()
num_iterations = 5000000
pbar = ProgressbarText3(num_iterations)
for i in range(1, num_iterations + 1):
    pbar.progress(i)
toc = time()
print()
elapsed_time = toc - tic
print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

***************** 5000000/5000000 ****************
Elapsed time:  2.25s
Time per iteration: 4.5094685554504397e-07


# ProgressBarIPython

In [12]:
tic = time()
num_iterations = 5000000
pbar = ProgressBarIPython(num_iterations)
for i in range(1, num_iterations + 1):
    pbar.progress(i)
toc = time()
elapsed_time = toc - tic
print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

Elapsed time:  2.37s
Time per iteration: 4.745389461517334e-07


# TQDM

In [13]:
tic = time()
num_iterations = 5000000
pbar = tqdm(total=num_iterations)
for i in range(num_iterations):
    pbar.update()
pbar.close()

toc = time()
elapsed_time = toc - tic
print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

100%|██████████| 5000000/5000000 [00:01<00:00, 2652476.96it/s]

Elapsed time:  1.89s
Time per iteration: 3.7746176719665527e-07


# ZMQ Progressbar

In [14]:
from pyphysim.progressbar.progressbar import ProgressbarZMQClient, ProgressbarZMQServer
from time import time, sleep
from pyphysim.util.misc import pretty_time

In [15]:
num_iterations = 5000000
# num_iterations = 100

server = ProgressbarZMQServer()

c1 = server.register_client_and_get_proxy_progressbar(num_iterations)
c2 = server.register_client_and_get_proxy_progressbar(num_iterations)

server.start_updater()

In [16]:
tic = time()

# c1 = ProgressbarZMQClient(0, 'localhost', 7396, num_iterations)
for i in range(1, num_iterations + 1):
    #     sleep(0.1)
    c1.progress(i)
toc = time()
elapsed_time = toc - tic

print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

[*********************  44%                      ]  Elapsed time:  2.25s
Time per iteration: 4.5099935531616213e-07


In [17]:
tic = time()

# c2 = ProgressbarZMQClient(0, 'localhost', 7396, num_iterations)
for i in range(1, num_iterations + 1):
    #     sleep(0.1)
    c2.progress(i)
toc = time()
elapsed_time = toc - tic

server.stop_updater()

print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

[**********************100%**********************]  
Elapsed time:  2.19s
Time per iteration: 4.383201599121094e-07


In [18]:
server.stop_updater()

# Mulriprocess Progressbar

In [25]:
from pyphysim.progressbar.progressbar import ProgressbarMultiProcessServer, ProgressbarMultiProcessClient
from time import time, sleep
from pyphysim.util.misc import pretty_time

In [38]:
num_iterations = 10000000
# num_iterations = 100

server = ProgressbarMultiProcessServer(message="{percent}% - Elapsed time: {elapsed_time}, ETA: {eta} ")

c1 = server.register_client_and_get_proxy_progressbar(num_iterations)
c2 = server.register_client_and_get_proxy_progressbar(num_iterations)

server.start_updater()

In [39]:
tic = time()

# c1 = ProgressbarZMQClient(0, 'localhost', 7396, num_iterations)
for i in range(1, num_iterations + 1):
    #     sleep(0.1)
    c2.progress(i)
toc = time()
elapsed_time = toc - tic

print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

[********************** 47%                      ]  47.356215% - Elapsed time: 4.33s, ETA: 4.81s A: 6.19s Elapsed time:  4.33s
Time per iteration: 4.329408407211304e-07


In [40]:
tic = time()

# c1 = ProgressbarZMQClient(0, 'localhost', 7396, num_iterations)
for i in range(1, num_iterations + 1):
    #     sleep(0.1)
    c2.progress(i)
toc = time()
elapsed_time = toc - tic

print()
print("Elapsed time: ", pretty_time(elapsed_time))
print(f"Time per iteration: {elapsed_time/num_iterations}")

[***********************50%                      ]  50.0% - Elapsed time: 5.42s, ETA: 5.42s 
Elapsed time:  1.33s
Time per iteration: 1.3264193534851073e-07


In [41]:
server.stop_updater()

[**********************100%**********************]  100.0% - Elapsed time: 6.45s, ETA: 0.00s 
